In [2]:
!pip install sentencepiece > /dev/null

In [3]:
import sentencepiece as spm

In [5]:
cd "/content/drive/My Drive/dataset/sentencepiece"

/content/drive/My Drive/dataset/sentencepiece


In [6]:
"""
spm.SentencePieceTrainer.Train(
    '--input=pre.txt, --model_prefix=sentencepiece_twitter --character_coverage=0.9995 --vocab_size=8000'
)
"""

In [7]:
sp = spm.SentencePieceProcessor()
sp.Load("sentencepiece_twitter.model")

True

In [14]:
sp.EncodeAsPieces("こんにちは、僕はゴリラです。")

['▁こんにちは', '、', '僕は', 'ゴリラ', 'です', '。']

In [16]:
import torchtext
from torchtext.data.utils import get_tokenizer

MAX_LENGTH = 30

#テキストに処理を行うFieldを定義
#fix_lengthはtokenの数
SRC = torchtext.data.Field(sequential=True, use_vocab=True, tokenize=sp.EncodeAsPieces,
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH,
                            eos_token='<eos>')

TRG = torchtext.data.Field(sequential=True, use_vocab=True, tokenize=sp.EncodeAsPieces,
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH,
                            init_token='<cls>', eos_token='<eos>')

train_ds, val_ds = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/dataset/TIU/twitter/5_20', train='train.csv', validation='val.csv', 
    format='csv', fields=[('src', SRC), ('trg', TRG)])

In [17]:
SRC.build_vocab(train_ds)
TRG.build_vocab(train_ds)
print(TRG.vocab.stoi)
print(len(TRG.vocab.stoi))

defaultdict(<function _default_unk_index at 0x7f0c181b3a60>, {'<unk>': 0, '<pad>': 1, '<cls>': 2, '<eos>': 3, '▁': 4, '。': 5, 'です': 6, '、': 7, '▁私': 8, 'は': 9, 'の': 10, 'た': 11, 'て': 12, 'も': 13, '!': 14, 'っ': 15, 'ね': 16, 'に': 17, 'し': 18, 'が': 19, 'か': 20, 'で': 21, 'な': 22, 'よ': 23, 'ます': 24, 'い': 25, 'ん': 26, 'と': 27, '...': 28, 'だ': 29, 'ない': 30, '?': 31, 'から': 32, '!!': 33, '目': 34, '▁ふ': 35, '口': 36, 'さん': 37, 'ござ': 38, '▁お': 39, 'を': 40, 'いい': 41, '▁ありがとう': 42, '笑': 43, 'てる': 44, 'まし': 45, 'w': 46, 'れ': 47, '▁そう': 48, 'けど': 49, 'ので': 50, 'さ': 51, 'や': 52, 'たら': 53, 'たい': 54, 'る': 55, '▁う': 56, 'って': 57, '好き': 58, 'ば': 59, '▁わ': 60, 'ちゃん': 61, 'き': 62, 'ら': 63, 'り': 64, '人': 65, '見': 66, '▁あ': 67, 'こと': 68, '!!!': 69, 'く': 70, '▁それ': 71, '嬉しい': 72, '▁大丈夫': 73, '▁え': 74, 'でも': 75, 'ください': 76, 'す': 77, '気': 78, '▁今日': 79, 'する': 80, '▁1': 81, '......': 82, '??': 83, 'こ': 84, 'www': 85, '▁これ': 86, 'とか': 87, '▁大': 88, 'なんて': 89, '▁み': 90, '呼': 91, '▁何': 92, 'せ': 93, 'ちゃ': 94, '方': 95,

In [20]:
from torchtext import data

batch_size = 128

train_dl = data.Iterator(train_ds, batch_size=batch_size, train=True)
val_dl = data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)
batch = next(iter(val_dl))
print(batch.trg[0])

tensor([[   2,    4, 1189,  ...,    1,    1,    1],
        [   2,  837,    4,  ...,    4,   18,    3],
        [   2,    4,    9,  ...,    4, 1244,    3],
        ...,
        [   2,  134,  151,  ...,    1,    1,    1],
        [   2,    8,    4,  ...,    1,    1,    1],
        [   2,   35,   11,  ...,    1,    1,    1]])
